<a href="https://colab.research.google.com/github/kweenkeen/ReachOutandRead/blob/main/Mail_Merge_Monthly_Email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import pandas as pd
from google.colab import files
from datetime import date
from datetime import datetime

# Make sure to upload site data report for TWO REPORTING PERIODS!

site_data_report = files.upload()
site_data_file = next(iter(site_data_report))
df = pd.read_excel(site_data_file, converters={'Site ID': str})
df = df.set_index('Site ID')

Saving MyROR Site Data Report 2022-07-15.xlsx to MyROR Site Data Report 2022-07-15 (1).xlsx


In [74]:
# Set filter
active_or_approved = ['Active', 'Active - Non-Reporting', 'Slated for deactivation', 'COVID-19 Freeze', 'Approved - Needs Training', 'Approved - Waitlisted', 'Complete - Awaiting Approval', 'In Progress'] 
df = df[df['Status'].isin(active_or_approved)] 

In [75]:
# UPLOAD AFFILIATE CONTACT LIST

affiliate_contact_list = files.upload()

Saving 2022.07.12 Find a Site Affiliate Contact Information.xlsx to 2022.07.12 Find a Site Affiliate Contact Information (1).xlsx


In [76]:
affiliate_filename = next(iter(affiliate_contact_list))

affiliate_contact_df = pd.read_excel(affiliate_filename)

df = df.join(affiliate_contact_df.set_index('Affiliate Name'), on='Affiliate Name')

In [77]:
df.head()

,Site Type,Site Name,Status,Legacy Satellite ID,Affiliate Name,Health System Name,Regions,Categories,Inactivated Reason,Physical Location: Line 1,...,AAB Date,Scholastic Account,Scholastic Balance,Scholastic Date,Most Recent Quality Classification Date,Most Recent Quality Classification Assessment,Most Recent Quality Classification Activity,"Site is encouraging parents/caregivers to talk, read, sing, and play with their babies at the WCV’s 0-5 months as instructed by the ROR anticipatory guidance",Site wants to give books at all or some WCVs from 0-5 months,Affiliate Contact
Site ID,,,,,,,,,,,,,,,,,,,,,
2,Primary,UAB Primary Care Clinic /,Active,NaN,Reach Out and Read Alabama,NaN,"Prescription for Summer Reading 2018, ACGLR - ...",NaN,NaN,1600 7th Avenue South,...,2022-07-14,2660370,2227.5,2022-07-11,2018-12-12,Green,2018-12-12,Y,Y,stsowell@roralabama.org
8,Primary,Quality of Life Health Services /,Active,NaN,Reach Out and Read Alabama,NaN,NaN,NaN,NaN,1411 Piedmont Cut-off,...,2022-07-14,4577849,0.0,2022-07-11,2018-06-01,Yellow,2018-06-01,Y,Y,stsowell@roralabama.org
24,Primary,Alaska Native Medical Center / PCC-Pediatric C...,Slated for deactivation,NaN,Independent / International,NaN,NaN,"ROR American Indian/Alaska Native, Grant Funde...",NaN,4320 Diplomacy Drive,...,NaT,7244523,0.0,2022-07-11,2017-08-02,Yellow,2017-08-02,NaN,NaN,NONE
29,Primary,Valleywise Community Health Center- Avondale /,Active,NaN,Reach Out and Read Arizona,NaN,"Southwest Human Development, AZ",NaN,NaN,950 E Van Buren Street,...,NaT,2693483,0.0,2022-07-11,2021-10-27,Yellow,2021-10-27,Y,Y,melanie@azaap.org
30,Primary,Valleywise Community Health Center- Chandler /,Active,NaN,Reach Out and Read Arizona,NaN,"Southwest Human Development, AZ",NaN,NaN,811 S Hamilton,...,NaT,4613947,0.0,2022-07-11,2021-10-22,Yellow,2021-10-22,Y,Y,melanie@azaap.org


In [78]:
rslt_df = df[['Affiliate Contact', 'Affiliate Name', 'Status', 'Join Date', 'Training %', 'Total Trained', 'Most Recent Quality Classification Date']]


In [79]:
rslt_df.head()

,Affiliate Contact,Affiliate Name,Status,Join Date,Training %,Total Trained,Most Recent Quality Classification Date
Site ID,,,,,,,
2,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1997-12-01,0.90,9.0,2018-12-12
8,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1998-02-01,0.50,1.0,2018-06-01
24,NONE,Independent / International,Slated for deactivation,1997-04-01,0.67,8.0,2017-08-02
29,melanie@azaap.org,Reach Out and Read Arizona,Active,2001-01-01,0.63,5.0,2021-10-27
30,melanie@azaap.org,Reach Out and Read Arizona,Active,2000-01-01,0.80,4.0,2021-10-22


In [80]:
join_date_range_min = input("Type in the start of your year (MM/DD/YYYY): ")
join_date_range_min = datetime.strptime(join_date_range_min, '%m/%d/%Y')
join_date_range_max = input("Type in the end of your year (MM/DD/YYYY): ")
join_date_range_max = datetime.strptime(join_date_range_max, '%m/%d/%Y')

Type in the start of your year (MM/DD/YYYY): 07/01/2022
Type in the end of your year (MM/DD/YYYY): 07/15/2022


In [81]:
rslt_df['Join Date'] = pd.to_datetime(rslt_df['Join Date'], format='%m/%d/%Y')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
join_date_df = rslt_df.loc[(rslt_df['Join Date'] >= join_date_range_min) & (rslt_df['Join Date'] <= join_date_range_max)]
join_date_df['Join Date'] = join_date_df['Join Date'].dt.date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [83]:
join_date_df.head()

,Affiliate Contact,Affiliate Name,Status,Join Date,Training %,Total Trained,Most Recent Quality Classification Date
Site ID,,,,,,,
2439.6,jessie@reachoutandreadco.org,Reach Out and Read Colorado,Active,2022-07-11,1.00,1.0,NaT
2988.2,Meghan.Blomberg@reachoutandread.org,Reach Out and Read Massachusetts,Active,2022-07-11,1.00,8.0,NaT
3973.2,Meghan.Blomberg@reachoutandread.org,Reach Out and Read Massachusetts,Active,2022-07-11,1.00,5.0,NaT
7136.3,dharrisschaap@readyforschool.org,Michigan (Mars Pathfinder),Active,2022-07-13,1.00,2.0,NaT
11989.2,NaN,Reach Out and Read Washington,Active,2022-07-05,0.83,5.0,NaT


In [84]:
join_date_df.head()

,Affiliate Contact,Affiliate Name,Status,Join Date,Training %,Total Trained,Most Recent Quality Classification Date
Site ID,,,,,,,
2439.6,jessie@reachoutandreadco.org,Reach Out and Read Colorado,Active,2022-07-11,1.00,1.0,NaT
2988.2,Meghan.Blomberg@reachoutandread.org,Reach Out and Read Massachusetts,Active,2022-07-11,1.00,8.0,NaT
3973.2,Meghan.Blomberg@reachoutandread.org,Reach Out and Read Massachusetts,Active,2022-07-11,1.00,5.0,NaT
7136.3,dharrisschaap@readyforschool.org,Michigan (Mars Pathfinder),Active,2022-07-13,1.00,2.0,NaT
11989.2,NaN,Reach Out and Read Washington,Active,2022-07-05,0.83,5.0,NaT


In [85]:
rslt_df.head()

,Affiliate Contact,Affiliate Name,Status,Join Date,Training %,Total Trained,Most Recent Quality Classification Date
Site ID,,,,,,,
2,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1997-12-01,0.90,9.0,2018-12-12
8,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1998-02-01,0.50,1.0,2018-06-01
24,NONE,Independent / International,Slated for deactivation,1997-04-01,0.67,8.0,2017-08-02
29,melanie@azaap.org,Reach Out and Read Arizona,Active,2001-01-01,0.63,5.0,2021-10-27
30,melanie@azaap.org,Reach Out and Read Arizona,Active,2000-01-01,0.80,4.0,2021-10-22


In [86]:
mail_merge_df = pd.DataFrame(columns=['Affiliate Name'])

In [87]:
mail_merge_df = mail_merge_df.set_index('Affiliate Name')

In [88]:
mail_merge_df.head()

""
Affiliate Name


In [89]:
# join_date_df = join_date_df['Affiliate Name'].value_counts()

In [90]:
high_level_active = ['Active', 'Active - Non-Reporting', 'Slated for deactivation', 'COVID-19 Freeze'] 
active_df = rslt_df[rslt_df['Status'].isin(high_level_active)] 

In [91]:
active_sites_df = active_df['Affiliate Name'].value_counts()

In [92]:
active_sites_df =active_sites_df.transpose()

In [93]:
active_sites_df.head()

Independent / International          656
Reach Out and Read North Carolina    375
Reach Out and Read Colorado          329
Reach Out and Read Minnesota         301
Reach Out and Read Massachusetts     268
Name: Affiliate Name, dtype: int64

In [94]:
mail_merge_df['Number of Sites (High Level Active)'] = active_sites_df

In [95]:
mail_merge_df.head()

,Number of Sites (High Level Active)
Affiliate Name,
Independent / International,656
Reach Out and Read North Carolina,375
Reach Out and Read Colorado,329
Reach Out and Read Minnesota,301
Reach Out and Read Massachusetts,268


In [96]:
new_sites_df = join_date_df['Affiliate Name'].value_counts()

In [97]:
mail_merge_df['Number of New Sites'] = new_sites_df

In [149]:
mail_merge_df['Number of New Sites'] = mail_merge_df['Number of New Sites'].fillna(0)

In [150]:
mail_merge_df['Number of New Sites'] = mail_merge_df['Number of New Sites'].astype(int)

In [98]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites
Affiliate Name,,
Independent / International,656,1.0
Reach Out and Read North Carolina,375,NaN
Reach Out and Read Colorado,329,1.0
Reach Out and Read Minnesota,301,1.0
Reach Out and Read Massachusetts,268,2.0


In [99]:
active_df.head()

,Affiliate Contact,Affiliate Name,Status,Join Date,Training %,Total Trained,Most Recent Quality Classification Date
Site ID,,,,,,,
2,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1997-12-01,0.90,9.0,2018-12-12
8,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1998-02-01,0.50,1.0,2018-06-01
24,NONE,Independent / International,Slated for deactivation,1997-04-01,0.67,8.0,2017-08-02
29,melanie@azaap.org,Reach Out and Read Arizona,Active,2001-01-01,0.63,5.0,2021-10-27
30,melanie@azaap.org,Reach Out and Read Arizona,Active,2000-01-01,0.80,4.0,2021-10-22


In [100]:
training_percent_average_df = active_df.groupby('Affiliate Name')['Training %'].mean()

In [101]:
training_percent_average_df.head()

Affiliate Name
Delaware (TIROS 1)                     0.220000
Idaho (Curiosity Rover)                0.760909
Independent / International            0.686371
Michigan (Mars Pathfinder)             0.788969
PA (excluding Greater Philadelphia)    0.826892
Name: Training %, dtype: float64

In [139]:
mail_merge_df['Current Overall Training Percentage'] = training_percent_average_df

In [140]:
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage']* 100

In [141]:
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage'].astype(int)

In [142]:
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage'].astype(str) + '%'

In [143]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained
Affiliate Name,,,,
Independent / International,656,1.0,68%,2333
Reach Out and Read North Carolina,375,NaN,93%,2001
Reach Out and Read Colorado,329,1.0,92%,1635
Reach Out and Read Minnesota,301,1.0,86%,2385
Reach Out and Read Massachusetts,268,2.0,98%,1823


In [144]:
providers_trained_count_df = active_df.groupby('Affiliate Name')['Total Trained'].sum().astype(int)

In [145]:
providers_trained_count_df.head()

Affiliate Name
Delaware (TIROS 1)                       17
Idaho (Curiosity Rover)                  95
Independent / International            2333
Michigan (Mars Pathfinder)              339
PA (excluding Greater Philadelphia)    1164
Name: Total Trained, dtype: int64

In [146]:
mail_merge_df['Number of Providers Trained'] = providers_trained_count_df

In [151]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained
Affiliate Name,,,,
Independent / International,656,1,68%,2333
Reach Out and Read North Carolina,375,0,93%,2001
Reach Out and Read Colorado,329,1,92%,1635
Reach Out and Read Minnesota,301,1,86%,2385
Reach Out and Read Massachusetts,268,2,98%,1823
